In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv(r"Iris.csv")
# 我们要通过花的三个属性预测第四个属性 所以Species & Id是多余的
data.drop(["Id", "Species"], axis=1, inplace=True)
# 删除重复记录
data.drop_duplicates(inplace=True)

In [23]:
class KNN:
    """"
    使用py 实现k近邻算法（回归预测）
    该算法用于回归预测 根据前三个特征属性 寻找最近的K个近邻 然后再根据k个近邻的第四个%save征属性
    预测样本的第四个特征值
    """
    def __init__(self, k):
        """初始化方法 
        
        Parameters: 
        -----------
        k : int 
            邻居的个数
        """
        self.k = k
    def fit(self, X, y):
        """训练方法
        
        Parameters:
        ------------
        X: 类数组类型（特征矩阵） 形状为（样本数量， 特征数量）
           待训练的样本特征（属性）
        y: 类数组类型（目标标签），形状为（样本数量）
           每个样本的目标值（标签）
        """    
        # 将X和y都转换成ndarray数组的形式 方便统一操作
        self.X = np.asarray(X)
        self.y = np.asarray(y)
        
    def predict(self, X):
        """根据参数传递的样本，预测样本数据
        Parameters
        ------
        X : 类数组类型 形状为:[样本数量，特征数量]
            待训练的样本特征(属性)
        Returns
        ------
        result : 数组类型
                预测的结果
        """
        X = np.asarray(X)
        result = []
        for x in X:
            # 计算距离 计算与训练集中每个X的距离
            dis = np.sqrt(np.sum((x - self.X) ** 2, axis=1))
            #返回数组排序后 每个元素在原数组中（排序之前的数组）的索引
            index = dis.argsort()
            # 取前k个距离最近的索引 原数组中的索引
            index = index[:self.k]
            #计算均值 然后返回结果列表
            result.append(np.mean(self.y[index]))
        return np.asarray(result)
    def predict2(self, X):
        """根据参数传递的样本，预测样本数据 !!带权重!!
        
        权重的计算方式：使用每个节点（邻居）距离的倒数 / 所有节点距离倒数之和
        
        Parameters
        ------
        X : 类数组类型 形状为:[样本数量，特征数量]
            待训练的样本特征(属性)
        Returns
        ------
        result : 数组类型
                预测的结果
        """
        X = np.asarray(X)
        result = []
        for x in X:
            # 计算距离 计算与训练集中每个X的距离
            dis = np.sqrt(np.sum((x - self.X) ** 2, axis=1))
            #返回数组排序后 每个元素在原数组中（排序之前的数组）的索引
            index = dis.argsort()
            # 取前k个距离最近的索引 原数组中的索引
            index = index[:self.k]
            # 求所有邻居节点距离的倒数之和 加一个小小值是为了避免距离为0
            s = np.sum(1 / (dis[index] + 0.001))
            weight = (1 / (dis[index] + 0.001)) / s
            #使用邻居节点的标签值 乘以对应的权重 然后相加求和得到最终的预测结果
            result.append(np.sum(self.y[index] * weight))
        return np.asarray(result)

In [36]:
t = data.sample(len(data), random_state=0)
train_X = t.iloc[:120, :-1]
train_y = t.iloc[:120, -1]
test_X = t.iloc[120:, :-1]
test_y = t.iloc[120:, -1]
knn = KNN(k=5)
knn.fit(train_X,train_y)
result1 = knn.predict(test_X)
result2 = knn.predict2(test_X)
display(result1) 
display(result2) 
# 取平方是为了避免正负误差抵消
display(test_y.values)
display(np.mean((result1 - test_y) ** 2))
display(np.mean((result2 - test_y) ** 2))

array([1.32, 1.94, 1.26, 1.26, 1.98, 1.22, 2.12, 0.3 , 1.92, 1.4 , 1.18,
       0.26, 1.88, 2.28, 1.72, 0.18, 1.1 , 1.34, 1.64, 1.24, 0.18, 0.22,
       0.2 , 2.06, 1.18, 1.82, 0.22])

array([1.33760032, 1.97520882, 1.19702048, 1.25718133, 2.19471225,
       1.1978922 , 2.10475545, 0.31428522, 1.9098273 , 1.38879853,
       1.1902747 , 0.24757247, 1.92343332, 2.25388136, 1.68334648,
       0.18046738, 1.07412341, 1.33225934, 1.74401891, 1.23506493,
       0.1807207 , 0.2308537 , 0.2       , 2.04520936, 1.17609431,
       1.83294464, 0.21704628])

array([1.5, 1.8, 1. , 1.3, 2.1, 1.2, 2.2, 0.2, 2.3, 1.3, 1. , 0.2, 1.6,
       2.1, 2.3, 0.3, 1. , 1.2, 1.5, 1.3, 0.2, 0.4, 0.1, 2.1, 1.1, 1.5,
       0.2])

0.037437037037037026

0.04058967860510454

In [27]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams["font.family"] = "SimHei"
mpl.rcParams["axes.unicode_minus"] = False

In [1]:
plt.figure(figsize=(10,10))
# 绘制预测值
plt.plot(result2,  "ro-", label="预测值")
# 绘制真实值
plt.plot(test_y.values, "go--", label="真实值")
plt.title("KNN回归 连续值预测展示")
plt.xlabel("节点序号")
plt.ylabel("花瓣宽度")
plt.legend()
plt.show()

NameError: name 'plt' is not defined